## Create cost surfaces 
This notebook creates a simple set of cost distance surfaces, one for each biogas source. 

In [17]:
#Import packages
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import osmnx as ox
from pyproj import CRS
import rasterio
from rasterio import features, plot
from matplotlib import pyplot as plt

### Read in the cost surface raster

In [2]:
#Load the cost surface raster
cost_surface_dataset = rasterio.open('../data/costsurface500m.img')
#Extract the first (and only) band
cost_surface = cost_surface_dataset.read(1)
#Extract the coordinate reference system
cost_crs = cost_surface_dataset.crs

In [ ]:
#Display 
plot.show(cost_surface_dataset,cmap='YlOrRd')

### Read in the biogas location data
These data are stored as the "Swine" sheet in the "NC_Biogas_Potential.xlsx" file. These records include the lat/long coordinates of each swine lagoon as well as its biogas potential in ($f^3/yr$). 
* First, calculations must be made to convert the potential into cost to pipe ($\$/{mi-MMBtu@15yr}$).
* Then the data must be converted to a geodataframe and transformed to match the coordinate reference system of the cost surface.

In [4]:
#Read in biogas sources (example: Duplin Co)
dfBG =  pd.read_excel('../data/NC_Biogas_Potential.xlsx',sheet_name='Swine')

In [5]:
dfBG.head(1)

,Facility Name,Address,City,County Name,Zip,Latitude,Longitude,Regulated Activity,Allowable Count,Total Waste (tons / year),Biogas Potential (m^3 / year),Biogas Potential (f^3 / year)
0,Piedmont L/S Co Farm,NaN,NaN,Alamance,NaN,36.1742,-79.4994,Swine - Farrow to Wean,300,3982.050316,111497.408837,3.937494e+06


#### Create a dataframe of per-yead yield by farm type

In [7]:
#Create a dataframe of per-yead yield by farm type
biogas_methane_content = 0.60 #B3 
biogas_CO2_content = 0.40     #B2
MMBtu_per_tsf = 1.037         #C6


#Create dataframe of type and biogas yield
df_farm_type_yield = pd.DataFrame(data = [['Swine - Feeder to Finish', 2336],
                                          ['Swine - Wean to Feeder', 420], 
                                          ['Swine - Wean to Finish', 2068], 
                                          ['Swine - Farrow to Wean', 1183],
                                          ['Swine - Farrow to Finish',1603]],
                                  columns = ['farm type','biogas (scf/y)'])

#Compute methane yield from biogas and constants
df_farm_type_yield['methane (MMBtu/year)'] = (df_farm_type_yield['biogas (scf/y)'] * 
                                              biogas_methane_content *
                                              MMBtu_per_tsf / 1000)

#Set the index to the farm type
df_farm_type_yield.set_index('farm type',inplace=True)

#Reveal the table
df_farm_type_yield

,biogas (scf/y),methane (MMBtu/year)
farm type,,
Swine - Feeder to Finish,2336,1.453459
Swine - Wean to Feeder,420,0.261324
Swine - Wean to Finish,2068,1.286710
Swine - Farrow to Wean,1183,0.736063
Swine - Farrow to Finish,1603,0.997387


#### Conversions
Construct a function that computes hourly methane yield for each lagoon based on its farm type and head count

In [31]:
#Conversion of animal count & type to MMBtu/hour
def get_methane_yield(animal_count, farm_type):
    #Constants
    hours_per_year = 365 * 24     #Hours in a year [C6]
    #Compute biogas yield feed flow (scf/h)
    lagoon_biogas_feed_flow = df_farm_type_yield.loc[farm_type,'biogas (scf/y)'] * animal_count / hours_per_year 
    #Compute methane yield MMBtu/yr
    lagoon_methane_yield_hr = df_farm_type_yield.loc[farm_type,'methane (MMBtu/year)'] * animal_count 
    #Convert methane yield to scf/h
    lagoon_methane_yield_yr = lagoon_methane_yield / hours_per_year * 1000 * MMBtu_per_tsf
    #Return the lagoon methane yield
    return (lagoon_biogas_feed_flow, lagoon_methane_yield)

In [32]:
#Conversion of methane yield to pipe diameter: in
def get_pipe_diameter(methane_yield):
    #Constants
    low_pressure_diameter_power = 0.394       #C62
    low_pressure_diameter_coefficient = 0.051 #C63
    #Compute pipe diameter from yield
    pipe_diameter = low_pressure_diameter_coefficient * (methane_yield**low_pressure_diameter_power)
    #Return the pipe diameter
    return pipe_diameter

In [33]:
#Computation of methane production cost: $/MMBtu
def get_pipe_cost(biogas_feed):
    #Constants
    cost_curve_slope = -7.43      #G21
    cost_curve_intercept = 69.508 #H21
    #Computation
    pipe_cost = cost_curve_slope * np.log(biogas_feed) + cost_curve_intercept
    return pipe_cost

In [37]:
biogas_feed, methane_yield = get_methane_yield(11016,'Swine - Feeder to Finish')
print(f"Methane yield: {methane_yield:.0f} MMBtu/yr")

pipe_diam = get_pipe_diameter(methane_yield)
print(f"Pipe diameter: {pipe_diam:.3f}")

pipe_cost = get_pipe_cost(biogas_feed)
print(f"Pipe cost: ${pipe_cost:.2f} $/MMBty")

cost_year = methane_yield * pipe_cost
print(f"Annual cost: ${cost_year}")

Methane yield: 16011 MMBtu/yr
Pipe diameter: 2.313
Pipe cost: $10.18 $/MMBty
Annual cost: $162944.8711137337


##### Calculate cost-to-pipe from potential * head * yield/head
1. Convert $ft^{3}/{year}$ to $scf/hr$
2. Compute cost from cost curve coefficients: $\$/{MMBtu} =  -7.43 * ln(scf/hr) + 69.508)$
3. Compute 

In [ ]:
#Set constants
biogas_methane_content = 0.6  #60 percent methane

cost_curve_slope = -7.43      #G21
cost_curve_intercetp = 69.508 #H21

In [ ]:
#Pipeline cost function (low pressure) - to convert flow rate to $/mile
low_power_pressure = 0.485
low_power_coefficient = 10829
MMBtu_per_CuFt = 1.037

In [ ]:
dfBG['Biogas Potential (MMBtu per year)'] = dfBG['Biogas Potential (f^3 / year)'] * MMBtu_per_CuFt

In [ ]:
dfBG['Yield per head scf/yr'] = dfBG['Regulated Activity'].apply(
    lambda x: lagoon_yields_scf_per_year[x])

In [ ]:
#Compute yield/head rate from farm type
df_TypeYields = pd.read_csv('../Data/raw/swine_farm_type_yeilds.csv')
df_TypeYields
#Compute _ from allowable count and methane yield/head (based on )

In [ ]:
#Remove the total row
dfBG = dfBG.loc[dfBG['Facility Name']!='TOTAL']
#Sort values in ascending order by biogas potential
dfBG = dfBG.sort_values(by='Biogas Potential (f^3 / year)',
                        ascending=False).reset_index()

In [ ]:
#Convert to geodataframe from lat/lng coordinates
geom = [Point(xy) for xy in zip(dfBG['Longitude'],dfBG['Latitude'])]
gdfBG = gpd.GeoDataFrame(dfBG,geometry=geom,crs=4326)

In [ ]:
#Subset points falling in Duplin County
duplin_co = ox.gdf_from_place('Duplin County, NC').unary_union
gdfBG = gdfBG.loc[gdfBG.geometry.within(duplin_co)]

In [ ]:
#Project to same crs as raster
gdfBG = gdfBG.to_crs(prj)

In [ ]:
#Subset records intersecting cost surface
bbox = box(*ds.bounds)
gdfBG = gdfBG.loc[gdfBG.geometry.within(bbox)]

In [ ]:
#Get x and y offsets for each point
gdfBG['idx'] = gdfBG['geometry'].apply(lambda g: ds.index(g.x,g.y))
gdfBG['xOffset'] = gdfBG['geometry'].apply(lambda g: ds.index(g.x,g.y)[1])
gdfBG['yOffset'] = gdfBG['geometry'].apply(lambda g: ds.index(g.x,g.y)[0])

In [ ]:
dfBG.head()

In [ ]:
#Calcuate cost from the index
gdfBG['cost'] =gdfBG['idx'].apply(lambda x: band1[x])

In [ ]:
#Drop the index column
gdfBG.drop(columns='idx',inplace=True)
gdfBG.to_file('foo.shp')

In [ ]:
#Create a function to sample points
def get_cost(pt_geom):
    #Get the index
    idx = ds.index(pt_geom.x,pt_geom.y)
    #Get the band value
    band = ds.read(1)
    try:
        value = band[idx]
        return value
    except:
        return -1

In [ ]:
#Apply the function
#gdfBG['cost'] = gdfBG['geometry'].apply(get_cost)

In [ ]:
#Plot
fig,ax = plt.subplots(1,1,figsize=(10,10))
plot.show(ds,ax=ax,cmap='RdYlGn')
gdfBG.plot(ax=ax,color='blue')

In [ ]:
gdfBG

In [ ]:
#Iterate through all biogas site rows and create a cost distance raster
cost_arrays = [] #Cost arrays
cd_arrays = []   #Cost distance arrays, min cost distance back to source
tb_arrays = []   #Traceback arrays, for creating the least cost path

for i,r in gdfBG.iterrows():
    xOffset = r['xOffset']    #get the array x coordinate of the biogas source
    yOffset = r['yOffset']    #get the array y coordinate of the biogas soruce
    z = r['cost'] / 1609.34   #Get the pipeline cost (5th col) & convert from $/mile to $/meter
    base_cost = r[3]          #Get the base production cost (4th col)
    
    #Create the minimum cost path (MCP) surface, setting the distance between pixels to cell sizes
    lc_graph = graph.MCP_Geometric(arrCost * z , sampling=(x_size,y_size))
    #Compute cost distances away from the biogas source coordinates
    cd_array,tb_array = lc_graph.find_costs(starts=([(yOffset, xOffset)]))
    #Add the initial cost to all cells
    cd_array = cd_array + base_cost
    #Add to stack of arrays
    cost_arrays.append(arrCost * z)
    cd_arrays.append(cd_array)
    tb_arrays.append(tb_array)
        
#Create a stack of all arrays
arrCostStack = np.stack(cost_arrays)
arrCostDistanceStack =np.stack(cd_arrays)
arrTracebackStack = np.stack(tb_arrays)

#Save the array stacks to file
np.save('../data/DuplinCostStack.npy',arrCostStack)
np.save('../data/DuplinStack.npy',arrCostDistanceStack)
np.save('../data/DuplinTraceStack.npy',arrTracebackStack)